In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from admm.agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from admm.models import NN, Dummy
from admm.utils import add_params, scale_params, subtract_params, average_params, sum_params
import torch

%matplotlib inline

In [4]:
a = torch.randn(500,500)
print(torch.norm(a,p='fro'))

tensor(500.4350)


### ADMM Global Consensus

In [2]:
rho = 1

# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)
lambdas = np.vstack([lam, -lam])

agents = [
    GlobalConsensus(
        rho=rho, 
        x_init=np.random.randn(2), 
        lam_init=lam
    ) 
    for lam in lambdas
]

# comopute initial average
avg = 0
for agent in agents:
    avg += agent.x/len(agents) 

# broadcast average to all agents
for agent in agents:
    agent.primal_average = avg
    print(agent.x)
print(f'average = {agents[0].primal_average}')

[ 0.49934709 -0.03778324]
[-0.66796746  0.41283368]
[ 0.80950284 -1.01054746]
[-0.57426436  0.68750329]
average = [0.01665453 0.01300157]


In [3]:
for i in range(50):
    
    # compute primal variables
    for agent in agents:
        agent.primal_update()

    # compute new global average
    avg = 0
    for agent in agents:
        avg += agent.x/len(agents) 
    
    # broadcast average to all agents
    for agent in agents:
        agent.primal_avg = avg

    # update dual variables    
    for agent in agents:
        agent.dual_update()

for i, agent in enumerate(agents):
    print(f'agent {i}: x = {agent.x}, lam = {agent.lam}')

agent 0: x = [-2.18478557e-09  5.83137276e-10], lam = [ 4.36957115e-09 -1.16627455e-09]
agent 1: x = [-1.83312191e-10 -7.79049348e-10], lam = [3.66624381e-10 1.55809870e-09]
agent 2: x = [ 2.18478557e-09 -5.83137276e-10], lam = [-4.36957115e-09  1.16627455e-09]
agent 3: x = [1.83312191e-10 7.79049348e-10], lam = [-3.66624382e-10 -1.55809870e-09]


### Event-Based ADMM Global Consensus

In [4]:
rho = 1
deltas = [1e-3]
t_max = 30

# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)*5
lambdas = np.vstack([lam, -lam])
x_init = np.random.randn(lambdas.shape[0],2)*5
initial_avg = np.mean(x_init, axis=0)


In [5]:
for delta in deltas:

    # Initialise Agents
    agents = [
        EventGlobalConsensus(
            N=len(lambdas), 
            rho=rho, 
            delta=delta,
            x_init=x, 
            lam_init=lam
        ) 
        for lam, x in zip(lambdas, x_init)
    ]

    # broadcast average to all agents
    for agent in agents:
        agent.primal_avg = initial_avg

    # Run event based ADMM
    comm = 0

    for t in range(t_max):
        
        for agent in agents:
            agent.primal_update()
        
        sum_of_res = 0
        for agent in agents:
            if agent.broadcast: 
                comm += 1
                sum_of_res += agent.residual
            
        # This is somehow updating all the agents
        agent.primal_avg += sum_of_res
        
        for agent in agents:
            agent.dual_update()
    
    accuracy = np.sum([np.linalg.norm(agent.x - agent.C, ord=1) for agent in agents])
    load = comm/(t_max*len(agents))
    print(f'Accuracy = {1-accuracy:.6f}, load = {load}')
    

Accuracy = 0.999892, load = 0.6333333333333333
